In [11]:
import numpy as np
from collections import deque
import numpy as np
import time

In [47]:
def matrix_to_tuple(matrix):
    return (matrix[0][0], matrix[0][1], matrix[0][2],
            matrix[1][0], matrix[1][1], matrix[1][2],
            matrix[2][0], matrix[2][1], matrix[2][2])

A = np.array([[1, 1, 2], [0, 1, 1], [0, -3, -2]])
B = np.array([[-2, 0, -1], [-5, 1, -1], [3, 0, 1]])

I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


# C is the inverse of A
# D is the inverse of B
C = np.array([[1, -4, -1], [0, -2, -1], [0, 3, 1]])
D = np.array([[1, 0, 1], [2, 1, 3], [-3, 0, -2]])

# we are trying to figure out whether A, B, C, and D can generate SL3_Z

In [48]:
def are_we_done_yet(my_matrix):
    return (my_matrix==I).all()

In [5]:
def apply_mat(mat, index):
    if index==0:
        return mat @ A
    elif index==1:
        return mat @ B
    elif index==2:
        return mat @ C
    elif index==3:
        return mat @ D
    assert(1==2)

def generate_successors(last_mat):
    if last_mat==0:
        return [0, 1, 3]
    elif last_mat==1:
        return [0, 1, 2]
    elif last_mat==2:
        return [1, 2, 3]
    elif last_mat==3:
        return [0, 2, 3]
    elif last_mat==4:
        return [0, 1, 2, 3]


In [49]:
# adapted from ChatGPT
visited = set()
start_time = time.time()

def bfs(start):
    
    visited.add(matrix_to_tuple(start @ A))
    visited.add(matrix_to_tuple(start @ B))
    # visited.add(matrix_to_tuple(start @ C))
    # visited.add(matrix_to_tuple(start @ D))
    
    start_tuple_A = start @ A
    start_tuple_B = start @ B
    # start_tuple_C = start @ C
    # start_tuple_D = start @ D

    queue = deque([[start_tuple_A, 0, "0"], [start_tuple_B, 1, "1"]])

    while queue:
        vertex = queue.popleft()
        cur_mat = vertex[0]

        if time.time() >= start_time + 1:
            print("Timed out")
            break

        if are_we_done_yet(cur_mat):
            print(vertex)
            print("We found it!")
            break

        for neighbor in [0, 1]:
            # neighbor will take values 0, 1
            new_mat = apply_mat(cur_mat, neighbor)
            new_tuple = matrix_to_tuple(new_mat)
            if not new_tuple in visited:
                visited.add(new_tuple)
                queue.append([new_mat, neighbor, vertex[2] + str(neighbor)])

bfs(A @ B @ A @ B @ A @ B @ A)

# bfs(np.array([[1, 0, 1], [0, -1, -1], [0, 1, 0]]))
# print(visited)
# ran this overnight until it crashed my computer, didn't find anything :(

[array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]), 1, '1']
We found it!


In [62]:
A @ B @ B @ A @ A @ B @ B @ A

array([[ 11, -48, -14],
       [  6, -25,  -7],
       [-12,  48,  13]])

In [42]:
len(visited)

52289

In [9]:
# iterative deepening
# code adapted from https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/

# TODO also try this with the visited set. Would need to pass the visited set through the method calls
# visited = set()
 
# A function to perform a Depth-Limited search
# from given source 'src'
def DLS(src,target,maxDepth):
    # print(src)
    if (src[0] == target).all(): 
        print(src)
        return True

    # If reached the maximum depth, stop recursing.
    if maxDepth <= 0: 
        return False

    # Recur for all the vertices adjacent to this vertex
    for i in generate_successors(src[1]):
        new_mat = apply_mat(src[0], i)
        # new_tuple = matrix_to_tuple(new_mat)
        # if not new_tuple in visited:
            # visited.add(new_tuple)
        if(DLS([new_mat, i, src[2] + str(i)], target, maxDepth-1)):
            # print([new_mat, i, src[2] + str(i)])
            return True
    return False

# IDDFS to search if target is reachable from v.
# It uses recursive DLS()
def IDDFS(src, target, maxDepth):

    # Repeatedly depth-limit search till the maximum depth
    i = 0
    while i<maxDepth:
        if (DLS(src, target, i)):
            return True
        i += 1
 
# Create a graph given in the above diagram
# target = np.array([[1, 0, 1], [0, -1, -1], [0, 1, 0]])
target = A @ B @ A @ B
maxDepth = 20000
src = [A, 4, ""]


if IDDFS(src, target, maxDepth) == True:
    print("Target is reachable from source within max depth")
else :
    print("Target is NOT reachable from source within max depth")
    
# print(visited)


Target is NOT reachable from source within max depth
{(1, 0, 0, 0, 1, 0, 0, 0, 1), (1, -4, -1, 0, -2, -1, 0, 3, 1), (-1, 1, 0, -2, 1, 0, 9, -3, 1), (-3, 1, 0, -1, 1, 1, 0, -3, -5)}


In [8]:
A @ B @ A @ B

array([[-1,  0,  0],
       [ 0, -1,  0],
       [ 6,  3,  1]])